In [1]:
!pip install pysastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 5.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import xml.etree.ElementTree as ET
import re
import numpy as np
import seaborn as sns
import warnings
import string
warnings.filterwarnings("ignore", 'This pattern has match groups')
import matplotlib.pyplot as plt
import matplotlib

In [3]:
version = 3
df_human = pd.read_csv(f"Dataset/Human_Annotated.csv", index_col="ID")
df_human.fillna("", inplace = True)

df_human_umumnt = df_human.drop(columns=["Umum"])
df_human_umumnt["COUNT CLASS"] = df_human_umumnt.drop(columns=["JUDUL", "ISI"]).sum(axis=1)
df_human = df_human_umumnt[df_human_umumnt["COUNT CLASS"] > 0].drop(columns=["COUNT CLASS"])
del df_human_umumnt

# Get from corpus

In [4]:
h_code = list(df_human.index)

tree = ET.parse('Raw_Corpus_Abid/Corpus.xml')
root = tree.getroot()
df_lst = []
for dok in root:
  lst = []
  for tag in dok:
    if tag.tag == "ID" and tag.text in h_code:
      lst.append(tag.text)
      h_code.remove(tag.text)
    elif tag.tag == "ID" and tag.text not in h_code:
      break
    elif tag.tag == "TAG":
      labels = []
      for label in tag:
        labels.append(label.text)
      lst.append(labels)
    elif tag.tag == "KELUHAN":
      lst.append(tag[0].text)
      lst.append(tag[1].text)
    elif tag.tag == "JAWABAN":
      lst.append(tag[0].text)
  if lst != []:
    df_lst.append(lst)
df_corpus = pd.DataFrame(data=df_lst, columns=["ID", "CLASS", "JUDUL", "ISI", "JAWABAN"]).set_index("ID")
del df_lst

# Improved Map


In [5]:
def mapping(s):
  temp = set()
  for key, val in map.items():
    if is_in(key.lower(), s.lower()):
      temp.update(val)
  return list(temp)

def is_in(kw, text):
  if (len(kw.split()) == 1):
    kw = " " + kw + " "
    text = " " + text + " "
  return kw in text

def mapping_2(s):
  res = {}
  for key, val in map.items():
    if is_in(key.lower(), s.lower()):
      for label in val:
        if label not in res:
          res[label] = 0
        res[label] += 1
  return get_top_3(res)

import statistics
def get_top_3(res):
    max_value = max(res.values())
    std_dev = statistics.stdev(res.values())
    print(std_dev)
    top = [(key, value) for key, value in res.items() if value >= (max_value - std_dev)]
    print(top)
    sorted_res = sorted(top, key=lambda x: x[1], reverse=True)

    # Get the top 3 keys
    return [item[0] for item in sorted_res[:3]]


In [6]:
map = {
    "Anak" : ["Anak"],
    "Anak Remaja" : ["Anak"],
    "Asi" : ["Kandungan"],
    "Laktasi" : ["Kandungan"],
    "Kandungan" : ["Kandungan"],
    "Fat Loss dan Fitness" : ["Gizi"],
    "Jantung" : ["Jantung"],
    "Psikiater" : ["Jiwa"],


    "Alergi Anak" : ["Anak"],
    "Gizi Anak" : ["Anak"],
    "Kebidanan Kandungan" : ["Kandungan"],    "Bidan Kandung" : ["Kandungan"],
    "Haid" : ["Kandungan"],
    "Bedah" : ["Bedah"],
    "Gigi Mulut" : ["Gigi"],
    "Gizi" : ["Gizi"],
    "Berat Badan Ideal" : ["Gizi"],
    "Kecantikan" : ["Kulit dan Kelamin"],     "Cantik" : ["Kulit dan Kelamin"],
    "Kulit Dan Kelamin" : ["Kulit dan Kelamin"],
    "Masalah Kulit" : ["Kulit dan Kelamin"],
    "Kesehatan Jiwa" : ["Jiwa"],    "Sehat Jiwa" : ["Jiwa"],
    "Psikiatri" : ["Jiwa"],
    "Mata" : ["Mata"],
    "Penyakit Dalam" : ["Penyakit Dalam"],    "Sakit Dalam" : ["Penyakit Dalam"],
    "Tht" : ["THT"],
    "Tulang" : ["Tulang"],

    "Flu Singapura" : ["THT"],
    "Influenza" : ["THT"],
    "Anemia" : ["Penyakit Dalam"],

    "Abses" : ["Bedah"],
    "Aids" : ["Penyakit Dalam"],
    "Alergi" : ["Kulit dan Kelamin"],
    "Alergi Dingin" : ["Kulit dan Kelamin"],
    "Alkohol" : ["Penyakit Dalam"],
    "Amandel" : ["THT"],
    "Amlodipine" : ["Penyakit Dalam"],
    "Anak" : ["Anak"],
    "Anemia Defisiensi Besi" : ["Penyakit Dalam"],
    "Angin Duduk" : ["Jantung"],
    "Artritis" : ["Penyakit Dalam"],
    "Asam Lambung" : ["Penyakit Dalam"],
    "Asam Urat" : ["Penyakit Dalam"],
    "Asma" : ["Anak", "Paru"],
    "Autisme" : ["Anak"],
    "Batu Empedu" : ["Bedah"],
    "Batu Ginjal" : ["Bedah"],
    "Batuk" : ["THT"],
    "Bau Mulut" : ["Penyakit Dalam", "Gigi"],
    "Bayi" : ["Anak"],
    "Bekas Luka" : ["Kulit dan Kelamin"],
    "Benjolan" : ["Bedah"],       "Benjol" : ["Bedah"],
    "Bersalin" : ["Kandungan"],   "Salin" : ["Kandungan"],
    "Biang Keringat" : ["Kulit dan Kelamin"],
    "Bibir" : ["Kulit dan Kelamin"],
    "Biduran" : ["Kulit dan Kelamin"],      "Bidur" : ["Kulit dan Kelamin"],
    "Bintitan" : ["Mata"],                  "Bintit" : ["Mata"],
    "Bipolar" : ["Jiwa"],
    "Bph" : ["Urologi", "Bedah"],
    "Bronkitis" : ["Paru"],
    "Buta Warna" : ["Mata"],
    "Cacar Air" : ["Kulit dan Kelamin"],
    "Campak" : ["Anak"],
    "Cedera" : ["Bedah"],
    "Dehidrasi" : ["Penyakit Dalam"],
    "Demam" : ["Penyakit Dalam"],
    "Demam Berdarah" : ["Penyakit Dalam"],      "Demam Darah" : ["Penyakit Dalam"],
    "Demam Pada Anak" : ["Anak"],
    "Depresi" : ["Jiwa"],
    "Dermatitis" : ["Kulit dan Kelamin"],
    "Diabetes" : ["Penyakit Dalam"],
    "Diabetes Tipe 1" : ["Penyakit Dalam"],
    "Diabetes Tipe 2" : ["Penyakit Dalam"],
    "Diare" : ["Penyakit Dalam"],
    "Diet" : ["Gizi"],
    "Disentri" : ["Penyakit Dalam"],
    "Eksim" : ["Kulit dan Kelamin"],
    "Endometriosis" : ["Kandungan"],
    "Flu" : ["THT"],
    "Gagal Ginjal" : ["Penyakit Dalam"],
    "Gagal Jantung" : ["Jantung"],
    "Gastritis" : ["Penyakit Dalam"],
    "Gastroenteritis" : ["Penyakit Dalam"],
    "Gatal" : ["Kulit dan Kelamin"],
    "Gigi" : ["Gigi"],
    "Glaukoma" : ["Mata"],
    "Gondok" : ["Penyakit Dalam"],
    "Gonore" : ["Kulit dan Kelamin"],
    "Hamil" : ["Kandungan"],
    "Hamil Anggur" : ["Kandungan"],
    "Hematuria" : ["Urologi", "Penyakit Dalam", "Kandungan"],
    "Hepatitis" : ["Penyakit Dalam"],
    "Hepatitis A" : ["Penyakit Dalam"],
    "Hepatitis B" : ["Penyakit Dalam"],
    "Hepatitis C" : ["Penyakit Dalam"],
    "Hernia" : ["Bedah"],
    "Herpes Genital" : ["Kulit dan Kelamin"],
    "Herpes Zoster" : ["Kulit dan Kelamin"],
    "Hidrosefalus" : ["Anak"],
    "Hipertensi" : ["Jantung"],
    "Hipertiroidisme" : ["Penyakit Dalam"],
    "Hipoglikemi" : ["Penyakit Dalam"],
    "Hipoglikemia" : ["Penyakit Dalam"],
    "Hipotensi" : ["Jantung"],
    "Hipotiroidisme" : ["Penyakit Dalam"],
    "Hiv" : ["Penyakit Dalam"],
    "Hpv" : ["Kandungan", "Kulit dan Kelamin"],
    "Impetigo" : ["Kulit dan Kelamin"],
    "Imunisasi" : ["Anak"],
    "Infeksi Gigi" : ["Gigi"],
    "Infeksi Ginjal" : ["Penyakit Dalam"],
    "Infeksi Jamur" : ["Kulit dan Kelamin"],
    "Infeksi Kulit" : ["Kulit dan Kelamin"],
    "Infeksi Mata" : ["Mata"],
    "Infeksi Rahim" : ["Kandungan"],
    "Infeksi Saluran Kemih" : ["Urologi"],    "Infeksi Salur Kemih" : ["Urologi"],
    "Infeksi Telinga" : ["THT"],
    "Infeksi Telinga Tengah" : ["THT"],
    "Infertilitas" : ["Kandungan"],
    "Insomnia" : ["Saraf", "Jiwa"],
    "Intim Laki-Laki" : ["Kulit dan Kelamin"],  "Intim Laki" : ["Kulit dan Kelamin"],
    "Ispa" : ["Paru", "Anak"],
    "Itp" : ["Penyakit Dalam"],
    "Jerawat" : ["Kulit dan Kelamin"],
    "Kanker Darah" : ["Penyakit Dalam", "Bedah"],
    "Kanker Kulit" : ["Kulit dan Kelamin"],
    "Kanker Paru-Paru" : ["Paru", "Bedah"],   "Kanker Paru" : ["Paru", "Bedah"],
    "Kanker Paru Paru" : ["Paru", "Bedah"],
    "Kanker Payudara" : ["Bedah", "Penyakit Dalam"],
    "Kanker Prostat" : ["Urologi", "Bedah"],
    "Kanker Rahim" : ["Kandungan", "Bedah"],
    "Kanker Serviks" : ["Bedah", "Kandungan"],
    "Kanker Tulang" : ["Tulang", "Bedah"],
    "Kanker Usus Besar" : ["Bedah", "Penyakit Dalam"],
    "Katarak" : ["Mata"],
    "Keguguran" : ["Kandungan"],        "Gugur" : ["Kandungan"],
    "Kehamilan" : ["Kandungan"],        "Hamil" : ["Kandungan"],
    "Kehamilan Ektopik" : ["Kandungan"],  "Hamil Ektopik" : ["Kandungan"],
    "Kelahiran" : ["Kandungan"],         "Lahir" : ["Kandungan"],
    "Kelenjar Getah Bening" : ["THT", "Bedah", "Penyakit Dalam"],
    "Keputihan" : ["Kandungan"],        "Putih" : ["Kandungan"],

    "Keringat" : ["Kulit dan Kelamin"],
    "Kesehatan Mental" : ["Jiwa"],      "Sehat Mental" : ["Jiwa"],
    "Keseleo" : ["Bedah"],
    "Ketombe" : ["Kulit dan Kelamin"],
    "Kista" : ["Kandungan"],
    "Kista Ovarium" : ["Kandungan"],
    "Kolesterol" : ["Jantung",  "Penyakit Dalam"],
    "Kolesterol Tinggi" : ["Jantung",  "Penyakit Dalam"],
    "Komedo" : ["Kulit dan Kelamin"],
    "Konstipasi" : ["Penyakit Dalam"],
    "Kontrasepsi" : ["Kandungan"],
    "Kram" : ["Tulang"],
    "Kudis" : ["Kulit dan Kelamin"],
    "Kulit" : ["Kulit dan Kelamin"],
    "Kulit Bersisik" : ["Kulit dan Kelamin"],     "Kulit Sisik" : ["Kulit dan Kelamin"],
    "Kurap" : ["Kulit dan Kelamin"],
    "Kutil" : ["Kulit dan Kelamin"],
    "Kutil Kelamin" : ["Kulit dan Kelamin"],
    "Kutu Rambut" : ["Kulit dan Kelamin"],
    "Lensa Kontak" : ["Mata"],
    "Lipoma" : ["Bedah"],
    "Luka Bakar" : ["Bedah"],
    "Lupus" : ["Penyakit Dalam"],
    "Maag" : ["Penyakit Dalam"],
    "Malaria" : ["Penyakit Dalam"],
    "Masturbasi" : ["Kulit dan Kelamin"],
    "Mata Merah" : ["Mata"],
    "Melahirkan" : ["Kandungan"],           "Lahir" : ["Kandungan"],
    "Menopause" : ["Kandungan"],
    "Menstruasi" : ["Kandungan"],
    "Menyusui" : ["Kandungan"],
    "Merencanakan Kehamilan" : ["Kandungan"],   "Rencana Hamil" : ["Kandungan"],
    "Merokok" : ["Paru"],         "Rokok" : ["Paru"],
    "Migrain" : ["Saraf"],
    "Mimisan" : ["THT"],
    "Miom" : ["Kandungan"],
    "Mual" : ["Penyakit Dalam"],
    "Muntah" : ["Penyakit Dalam"],
    "Nutrisi" : ["Gizi"],
    "Nyeri Dada" : ["Jantung"],
    "Nyeri Haid" : ["Kandungan"],


    "Nyeri Sendi" : ["Penyakit Dalam", "Tulang"],
    "Operasi" : ["Bedah"],
    "Orang Tua" : ["Penyakit Dalam"],
    "Osteoarthritis" : ["Tulang"],
    "Osteoporosis" : ["Tulang"],
    "Ovulasi" : ["Kandungan"],
    "Panu" : ["Kulit dan Kelamin"],
    "Pasangan" : ["Kandungan"],       "Pasang" : ["Kandungan"],
    "Patah Tulang" : ["Tulang"],
    "Payudara" : ["Bedah"],
    "Pemutihan Kulit" : ["Kulit dan Kelamin"],              "Putih Kulit" : ["Kulit dan Kelamin"],
    "Penyakit Ginjal" : ["Penyakit Dalam"],                 "Sakit Ginjal" : ["Penyakit Dalam"],
    "Penyakit Jantung" : ["Jantung"],                       "Sakit Jantung" : ["Jantung"],
    "Penyakit Kuning" : ["Penyakit Dalam"],                 "Sakit Kuning" : ["Penyakit Dalam"],
    "Penyakit Mata" : ["Mata"],                             "Sakit Mata" : ["Mata"],
    "Penyakit Menular Seksual" : ["Kulit dan Kelamin"],     "Sakit Tular Seksual" : ["Kulit dan Kelamin"],
    "Penyakit Radang Pinggul" : ["Tulang"],                 "Sakit Radang Pinggul" : ["Tulang"],
    "Perut Kembung" : ["Penyakit Dalam"],
    "Pilek" : ["THT"],
    "Pingsan" : ["Penyakit Dalam",  "Jantung", "Saraf"],
    "Plasenta Previa" : ["Kandungan"],
    "Pneumonia" : ["Anak", "Paru"],
    "Polip" : ["THT"],
    "Ppok" : ["Paru"],
    "Psoriasis" : ["Kulit dan Kelamin"],
    "Rabun Jauh" : ["Mata"],
    "Radang Gusi" : ["Gigi"],
    "Radang Usus" : ["Penyakit Dalam"],
    "Rambut" : ["Kulit dan Kelamin"],
    "Rambut Rontok" : ["Kulit dan Kelamin"],
    "Rematik Artritis" : ["Tulang"],
    "Rheumatoid Arthritis" : ["Tulang"],
    "Rhinitis" : ["THT"],
    "Ruam Kulit" : ["Kulit dan Kelamin"],
    "Rubella" : ["Penyakit Dalam"],
    "Sakit Gigi" : ["Gigi"],
    "Sakit Kaki" : ["Tulang"],
    "Sakit Lutut" : ["Tulang"],
    "Sakit Maag" : ["Penyakit Dalam"],
    "Sakit Perut" : ["Penyakit Dalam"],
    "Sakit Pinggang" : ["Penyakit Dalam"],
    "Sakit Telinga" : ["THT"],
    "Sakit Tenggorokan" : ["THT"],
    "Hubungan Intim" : ["Kandungan"],       "Hubung Intim" : ["Kandungan"],
    "Serangan Jantung" : ["Jantung"],       "Serang Jantung" : ["Jantung"],
    "Serangan Panik" : ["Jiwa"],            "Serang Panik" : ["Jiwa"],
    "Sesak Napas" : ["Paru"],
    "Sifilis" : ["Kulit dan Kelamin"],
    "Sinusitis" : ["THT"],
    "Sirosis" : ["Penyakit Dalam"],
    "Skizofrenia" : ["Jiwa"],
    "Skoliosis" : ["Tulang"],
    "Stres" : ["Jiwa"],
    "Sukralfat" : ["Penyakit Dalam"],
    "Tahi Lalat" : ["Kulit dan Kelamin"],
    "Telinga" : ["THT"],
    "Telinga Berdenging" : ["THT"],         "Telinga Denging" : ["THT"],
    "Tetanus" : ["Penyakit Dalam"],
    "Thalassemia" : ["Anak"],
    "Tifus" : ["Penyakit Dalam"],
    "Tonsilitis" : ["THT"],
    "Trigliserida" : ["Penyakit Dalam"],
    "Trikomoniasis" : ["Kulit dan Kelamin"],
    "Tuberkulosis" : ["Penyakit Dalam"],
    "Tukak Lambung" : ["Penyakit Dalam"],
    "Tumor" : ["Bedah"],
    "Tumor Otak" : ["Saraf", "Bedah"],
    "Tumor Payudara" : ["Bedah"],
    "Usus Buntu" : ["Bedah"],
    "Vagina" : ["Kulit dan Kelamin", "Kandungan"],
    "Vaginitis" : ["Kulit dan Kelamin"],
    "Vaginosis" : ["Kulit dan Kelamin"],
    "Vaksin" : ["Anak"],
    "Vaksinasi" : ["Anak"],
    "Varikokel" : ["Urologi", "Bedah"],
    "Varises" : ["Jantung"],
    "Vitamin A" : ["Mata"],
    "Vitamin C" : ["Kulit dan Kelamin"],
    "Vitamin E" : ["Kulit dan Kelamin"],
    "Vitiligo" : ["Kulit dan Kelamin"],
    "Wajah" : ["Penyakit Dalam"],
    "Wasir" : ["Bedah"],
}

# Cleaning Corpus

In [7]:
import re
df_corpus["JUDUL"] = [re.sub(r'[^\w]', ' ', str(s).lower()) for s in df_corpus["JUDUL"].values]
df_corpus["ISI"] = [re.sub(r'[^\w]', ' ', str(s).lower()) for s in df_corpus["ISI"].values]

In [8]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
factory = StopWordRemoverFactory()
# stopwords = factory.get_stop_words()
# ext_stopwords = ['dok', 'doc', 'dokter', 'terima', 'kasih', 'terimakasih']
# whitelist = ['kecil', 'besar', 'mata', 'masalah', 'belakang']
# all_stopwords = list((set(stopwords) | set(ext_stopwords)) - set(whitelist))
all_stopwords = ["di", 'pada', 'ingin', 'dan', 'yang', 'sangat', 'luar', 'biasa', 'bagi', 'bagian', 'saat', 'hanya', 'asa', 'terasa', 'saat', 'dengan', 'bikin', 'dari', 'dalam', 'sering', 'saya', 'untuk']
dictionary = ArrayDictionary(all_stopwords)
stopword = StopWordRemover(dictionary)
df_corpus["JUDUL"] = [stopword.remove(s) for s in df_corpus["JUDUL"]]
df_corpus["ISI"] = [stopword.remove(s) for s in df_corpus["ISI"]]

In [9]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()
ignore = ["menyusui"]
df_corpus["JUDUL"] = [" ".join([stemmer.stem(w) if w not in ignore else w for w in str(s).split()]) for s in df_corpus["JUDUL"]]
df_corpus["ISI"] = [" ".join([stemmer.stem(w) if w not in ignore else w for w in str(s).split()]) for s in df_corpus["ISI"]]

In [10]:
df_corpus["CLASS_JUDUL"] = [mapping(s) for s in df_corpus["JUDUL"]]
df_corpus["CLASS_ISI"] = [mapping(s) for s in df_corpus["ISI"]]
df_corpus["CLASS_JUDUL_ISI"] = [mapping((j + " SEP " + i)) for j, i in df_corpus[["JUDUL", "ISI"]].values]
df_corpus["NEW_CLASS"] = [mapping(" , ".join(s)) for s in df_corpus["CLASS"]]

In [11]:
df_corpus.head(2)

,CLASS,JUDUL,ISI,JAWABAN,CLASS_JUDUL,CLASS_ISI,CLASS_JUDUL_ISI,NEW_CLASS
ID,,,,,,,,
DS-1,[other],mengapa keringat badan lebih,tanya konsultasi selamat malam dokter fathurro...,selamat malam Fathurrosi. Maaf berapa berat ba...,[Kulit dan Kelamin],[Kulit dan Kelamin],[Kulit dan Kelamin],[]
DS-23,[other],suka menghayal halusinasi,tanya konsultasi selamat siang dokter kali kha...,"Bila berkelanjutan, bisa saja memiliki ganggua...",[],[],[],[]


In [12]:
print("Jumlah instances CLASS_JUDUL yang bukan null: " + str(len(df_corpus[df_corpus["CLASS_JUDUL"].str.len() > 0])))
print("Jumlah instances CLASS_JUDUL yang null: " + str(len(df_corpus[df_corpus["CLASS_JUDUL"].str.len() == 0])))
print("Jumlah instances CLASS_ISI yang bukan null: " + str(len(df_corpus[df_corpus["CLASS_ISI"].str.len() > 0])))
print("Jumlah instances CLASS_ISI yang null: " + str(len(df_corpus[df_corpus["CLASS_ISI"].str.len() == 0])))
print("Jumlah instances CLASS_JUDUL_ISI yang bukan null: " + str(len(df_corpus[df_corpus["CLASS_JUDUL_ISI"].str.len() > 0])))
print("Jumlah instances CLASS_JUDUL_ISI yang null: " + str(len(df_corpus[df_corpus["CLASS_JUDUL_ISI"].str.len() == 0])))

Jumlah instances CLASS_JUDUL yang bukan null: 1104
Jumlah instances CLASS_JUDUL yang null: 455
Jumlah instances CLASS_ISI yang bukan null: 1325
Jumlah instances CLASS_ISI yang null: 234
Jumlah instances CLASS_JUDUL_ISI yang bukan null: 1385
Jumlah instances CLASS_JUDUL_ISI yang null: 174


# Compare Metric

In [13]:
from sklearn.metrics import classification_report, accuracy_score, jaccard_score, \
hamming_loss, cohen_kappa_score, f1_score, precision_score, recall_score
import math
def eval_all(y_human, y_machine, mlb):
  print("================ Classification Report ================")
  print(classification_report(y_human, y_machine, target_names=mlb.classes_, digits=4))
  print("================ Multi Label Score ================")
  eval_multi_label(y_human, y_machine, mlb)
  print("================ Cohen Kappa ================")
  cohen_kappa(y_human, y_machine, mlb)

def eval_multi_label(y_test, prediction, mlb):
    # Butuh informasi lebih lengkap? silakan disimak di bawah ini
    print('Accuracy             :', accuracy_score(y_test, prediction))
    print('Precision            :', precision_score(y_test, prediction, average='micro'))
    print('Recall               :', recall_score(y_test, prediction, average='micro'))
    print('F1 Score             :', f1_score(y_test, prediction, average='micro'))
    print('Hamming Loss         :', hamming_loss(y_test, prediction))
    print('Jaccard Score')
    js = jaccard_score(y_test, prediction, average=None)
    for i, j in zip(mlb.classes_, js):
      print(f"{i:<20} : {j}")
    print('Jaccard Score Macro Average:', jaccard_score(y_test, prediction, average='macro'))
    print('Jaccard Score Micro Average:', jaccard_score(y_test, prediction, average='micro'))

def cohen_kappa(y_human, y_test, mlb):
  sum = 0
  count = 0
  for i in range(len(mlb.classes_)):
    score = cohen_kappa_score(y_human[:,i], y_test[:,i])
    if not math.isnan(score):
      sum += score
      count += 1
    print(f"Cohen Kappa {mlb.classes_[i]:<20} : {cohen_kappa_score(y_human[:,i], y_test[:,i])}")
  print("Average Cohen Kappa              : " + str(sum/count))

## By Metadata

In [14]:
df = pd.merge(df_corpus.drop(columns=["JUDUL", "ISI", "JAWABAN", "CLASS_JUDUL", "CLASS_ISI", "CLASS_JUDUL_ISI", "CLASS"]),
                       df_human.drop(columns=["JUDUL", "ISI"]),
                       on="ID")
class_col = "NEW_CLASS"
df = df[df['NEW_CLASS'].map(lambda d: len(d)) > 0]
df

,NEW_CLASS,Anak,Bedah,Gigi,Gizi,Jantung,Jiwa,Kandungan,Kulit dan Kelamin,Mata,Paru,Penyakit Dalam,Saraf,THT,Tulang,Urologi
ID,,,,,,,,,,,,,,,,
AD-47,[Kandungan],0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
AD-117,"[Bedah, Penyakit Dalam]",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
AD-148,[Kulit dan Kelamin],0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
AD-179,"[Penyakit Dalam, Kandungan]",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
AD-233,"[Penyakit Dalam, Jantung]",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KD-43177,[Penyakit Dalam],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
KD-43189,[THT],0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
KD-43201,[Bedah],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb_std = MultiLabelBinarizer()
mlb_std.fit(df[class_col].values)

MultiLabelBinarizer()

In [16]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
y_human = df.drop(columns=[class_col]).values
y_machine = mlb_std.transform(df[class_col].values)

In [17]:
eval_all(y_human, y_machine, mlb_std)

================ Classification Report ================
                   precision    recall  f1-score   support

             Anak     0.7687    0.7984    0.7833       129
            Bedah     0.4545    0.6140    0.5224       114
             Gigi     0.9310    0.8438    0.8852        32
             Gizi     0.6627    0.5556    0.6044        99
          Jantung     0.1500    0.2727    0.1935        22
             Jiwa     0.8250    0.7765    0.8000        85
        Kandungan     0.8632    0.8417    0.8523       240
Kulit dan Kelamin     0.7053    0.8639    0.7766       169
             Mata     1.0000    0.8986    0.9466        69
             Paru     0.5417    0.7222    0.6190        18
   Penyakit Dalam     0.5219    0.7409    0.6124       193
            Saraf     0.4000    0.1395    0.2069        43
              THT     0.8864    0.8667    0.8764       135
           Tulang     0.5455    0.5854    0.5647        41
          Urologi     0.2500    0.1515    0.1887        33

## By Judul

In [18]:
df = pd.merge(df_corpus.drop(columns=["JUDUL", "ISI", "JAWABAN", "CLASS_ISI", "CLASS_JUDUL_ISI", "CLASS", "NEW_CLASS"]),
                       df_human.drop(columns=["JUDUL", "ISI"]),
                       on="ID")
df = df[df["CLASS_JUDUL"].str.len() > 0]
class_col = "CLASS_JUDUL"
df

,CLASS_JUDUL,Anak,Bedah,Gigi,Gizi,Jantung,Jiwa,Kandungan,Kulit dan Kelamin,Mata,Paru,Penyakit Dalam,Saraf,THT,Tulang,Urologi
ID,,,,,,,,,,,,,,,,
DS-1,[Kulit dan Kelamin],0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
DS-176,[Bedah],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
DS-324,[Bedah],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
AD-117,[Kandungan],0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
AD-148,[Kulit dan Kelamin],0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KD-43064,[Kulit dan Kelamin],0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
KD-43177,[Bedah],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
KD-43189,[THT],0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [19]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
y_human = df.drop(columns=[class_col]).values
y_machine = mlb_std.transform(df[class_col].values)

In [20]:
eval_all(y_human, y_machine, mlb_std)

================ Classification Report ================
                   precision    recall  f1-score   support

             Anak     0.7516    0.7423    0.7469       163
            Bedah     0.4869    0.7815    0.6000       119
             Gigi     0.9615    0.8929    0.9259        28
             Gizi     0.7273    0.1818    0.2909        44
          Jantung     0.4286    0.9000    0.5806        30
             Jiwa     0.9429    0.5238    0.6735        63
        Kandungan     0.8042    0.8616    0.8319       224
Kulit dan Kelamin     0.6582    0.8525    0.7429       183
             Mata     0.9028    0.9848    0.9420        66
             Paru     0.6000    0.4615    0.5217        13
   Penyakit Dalam     0.5000    0.4715    0.4854       123
            Saraf     0.2000    0.0400    0.0667        25
              THT     0.8898    0.8968    0.8933       126
           Tulang     0.6176    0.6364    0.6269        33
          Urologi     0.6667    0.1000    0.1739        20

## By Isi

In [21]:
df = pd.merge(df_corpus.drop(columns=["JUDUL", "ISI", "JAWABAN", "CLASS_JUDUL", "CLASS_JUDUL_ISI", "CLASS", "NEW_CLASS"]),
                       df_human.drop(columns=["JUDUL", "ISI"]),
                       on="ID")
df = df[df["CLASS_ISI"].str.len() > 0]
class_col = "CLASS_ISI"
df

,CLASS_ISI,Anak,Bedah,Gigi,Gizi,Jantung,Jiwa,Kandungan,Kulit dan Kelamin,Mata,Paru,Penyakit Dalam,Saraf,THT,Tulang,Urologi
ID,,,,,,,,,,,,,,,,
DS-1,[Kulit dan Kelamin],0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
DS-176,[Bedah],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
DS-277,"[Saraf, Paru, Jiwa]",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
DS-324,"[Bedah, Kulit dan Kelamin]",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
AD-27,[Anak],0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KD-43064,"[Penyakit Dalam, Kulit dan Kelamin]",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
KD-43177,"[Bedah, Kandungan]",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
KD-43189,"[THT, Jantung]",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [22]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
y_human = df.drop(columns=[class_col]).values
y_machine = mlb_std.transform(df[class_col].values)

In [23]:
eval_all(y_human, y_machine, mlb_std)

================ Classification Report ================
                   precision    recall  f1-score   support

             Anak     0.5979    0.9553    0.7355       179
            Bedah     0.3838    0.8583    0.5304       127
             Gigi     0.7857    0.9429    0.8571        35
             Gizi     0.5217    0.3333    0.4068        72
          Jantung     0.2560    0.9412    0.4025        34
             Jiwa     0.7636    0.5122    0.6131        82
        Kandungan     0.5831    0.9064    0.7097       267
Kulit dan Kelamin     0.5404    0.9062    0.6770       192
             Mata     0.6372    0.9863    0.7742        73
             Paru     0.4615    0.6316    0.5333        19
   Penyakit Dalam     0.3962    0.6526    0.4930       190
            Saraf     0.1429    0.0909    0.1111        44
              THT     0.7586    0.9103    0.8276       145
           Tulang     0.3735    0.7209    0.4921        43
          Urologi     0.5000    0.0345    0.0645        29

## By Judul + Isi

In [24]:
df = pd.merge(df_corpus.drop(columns=["JUDUL", "ISI", "JAWABAN", "CLASS_JUDUL", "CLASS_ISI", "CLASS", "NEW_CLASS"]),
                       df_human.drop(columns=["JUDUL", "ISI"]),
                       on="ID")
df = df[df["CLASS_JUDUL_ISI"].str.len() > 0]
class_col = "CLASS_JUDUL_ISI"
df

,CLASS_JUDUL_ISI,Anak,Bedah,Gigi,Gizi,Jantung,Jiwa,Kandungan,Kulit dan Kelamin,Mata,Paru,Penyakit Dalam,Saraf,THT,Tulang,Urologi
ID,,,,,,,,,,,,,,,,
DS-1,[Kulit dan Kelamin],0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
DS-176,[Bedah],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
DS-277,"[Saraf, Paru, Jiwa]",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
DS-324,"[Bedah, Kulit dan Kelamin]",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
AD-27,[Anak],0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KD-43064,"[Penyakit Dalam, Kulit dan Kelamin]",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
KD-43177,"[Bedah, Kandungan]",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
KD-43189,"[THT, Jantung]",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [25]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
y_human = df.drop(columns=[class_col]).values
y_machine = mlb_std.transform(df[class_col].values)

In [26]:
eval_all(y_human, y_machine, mlb_std)

================ Classification Report ================
                   precision    recall  f1-score   support

             Anak     0.5927    0.9781    0.7381       183
            Bedah     0.3758    0.8519    0.5215       135
             Gigi     0.7907    0.9444    0.8608        36
             Gizi     0.5577    0.3919    0.4603        74
          Jantung     0.2748    0.9730    0.4286        37
             Jiwa     0.7903    0.5568    0.6533        88
        Kandungan     0.5894    0.9278    0.7209       277
Kulit dan Kelamin     0.5347    0.9204    0.6764       201
             Mata     0.6496    0.9870    0.7835        77
             Paru     0.4643    0.6500    0.5417        20
   Penyakit Dalam     0.3940    0.6600    0.4935       200
            Saraf     0.1290    0.0889    0.1053        45
              THT     0.7667    0.9262    0.8389       149
           Tulang     0.3721    0.6957    0.4848        46
          Urologi     0.5000    0.0625    0.1111        32

# V2 Mapping

## Map2

In [27]:
map = {
    "Anak" : ["Anak"],
    "Anak Remaja" : ["Anak"],
    "Asi" : ["Kandungan"],
    "Laktasi" : ["Kandungan"],
    "Kandungan" : ["Kandungan"],
    "Fat Loss dan Fitness" : ["Gizi"],
    "Jantung" : ["Jantung"],
    "Psikiater" : ["Jiwa"],


    "Alergi Anak" : ["Anak"],
    "Gizi Anak" : ["Anak"],
    "Kebidanan Kandungan" : ["Kandungan"],    "Bidan Kandung" : ["Kandungan"],
    "Haid" : ["Kandungan"],
    "Bedah" : ["Bedah"],
    "Gigi Mulut" : ["Gigi"],
    "Gizi" : ["Gizi"],
    "Berat Badan Ideal" : ["Gizi"],
    "Kecantikan" : ["Kulit dan Kelamin"],     "Cantik" : ["Kulit dan Kelamin"],
    "Kulit Dan Kelamin" : ["Kulit dan Kelamin"],
    "Masalah Kulit" : ["Kulit dan Kelamin"],
    "Kesehatan Jiwa" : ["Jiwa"],    "Sehat Jiwa" : ["Jiwa"],
    "Psikiatri" : ["Jiwa"],
    "Mata" : ["Mata"],
    "Penyakit Dalam" : ["Penyakit Dalam"],    "Sakit Dalam" : ["Penyakit Dalam"],
    "Tht" : ["THT"],
    "Tulang" : ["Tulang"],

    "Flu Singapura" : ["THT"],
    "Influenza" : ["THT"],
    "Anemia" : ["Penyakit Dalam"],

    "Abses" : ["Bedah"],
    "Aids" : ["Penyakit Dalam"],
    "Alergi" : ["Kulit dan Kelamin"],
    "Alergi Dingin" : ["Kulit dan Kelamin"],
    "Alkohol" : ["Penyakit Dalam"],
    "Amandel" : ["THT"],
    "Amlodipine" : ["Penyakit Dalam"],
    "Anak" : ["Anak"],
    "Anemia Defisiensi Besi" : ["Penyakit Dalam"],
    "Angin Duduk" : ["Jantung"],
    "Artritis" : ["Penyakit Dalam"],
    "Asam Lambung" : ["Penyakit Dalam"],
    "Asam Urat" : ["Penyakit Dalam"],
    "Asma" : ["Anak", "Paru"],
    "Autisme" : ["Anak"],
    "Batu Empedu" : ["Bedah"],
    "Batu Ginjal" : ["Bedah"],
    "Batuk" : ["THT"],
    "Bau Mulut" : ["Penyakit Dalam", "Gigi"],
    "Bayi" : ["Anak"],
    "Bekas Luka" : ["Kulit dan Kelamin"],
    "Benjolan" : ["Bedah"],       "Benjol" : ["Bedah"],
    "Bersalin" : ["Kandungan"],   "Salin" : ["Kandungan"],
    "Biang Keringat" : ["Kulit dan Kelamin"],
    "Bibir" : ["Kulit dan Kelamin"],
    "Biduran" : ["Kulit dan Kelamin"],      "Bidur" : ["Kulit dan Kelamin"],
    "Bintitan" : ["Mata"],                  "Bintit" : ["Mata"],
    "Bipolar" : ["Jiwa"],
    "Bph" : ["Urologi", "Bedah"],
    "Bronkitis" : ["Paru"],
    "Bronchitis"  : ["Paru"],
    "Buta Warna" : ["Mata"],
    "Cacar Air" : ["Kulit dan Kelamin"],
    "Campak" : ["Anak"],
    "Cedera" : ["Bedah"],
    "Dehidrasi" : ["Penyakit Dalam"],
    "Demam" : ["Penyakit Dalam"],
    "Demam Berdarah" : ["Penyakit Dalam"],      "Demam Darah" : ["Penyakit Dalam"],
    "Demam Pada Anak" : ["Anak"],
    "Depresi" : ["Jiwa"],
    "Dermatitis" : ["Kulit dan Kelamin"],
    "Diabetes" : ["Penyakit Dalam"],
    "Diabetes Tipe 1" : ["Penyakit Dalam"],
    "Diabetes Tipe 2" : ["Penyakit Dalam"],
    "Diare" : ["Penyakit Dalam"],
    "Diet" : ["Gizi"],
    "Disentri" : ["Penyakit Dalam"],
    "Eksim" : ["Kulit dan Kelamin"],
    "Endometriosis" : ["Kandungan"],
    "Flu" : ["THT"],
    "Gagal Ginjal" : ["Penyakit Dalam"],
    "Gagal Jantung" : ["Jantung"],
    "Gastritis" : ["Penyakit Dalam"],
    "Gastroenteritis" : ["Penyakit Dalam"],
    "Gatal" : ["Kulit dan Kelamin"],
    "Gigi" : ["Gigi"],
    "Glaukoma" : ["Mata"],
    "Gondok" : ["Penyakit Dalam"],
    "Gonore" : ["Kulit dan Kelamin"],
    "Hamil" : ["Kandungan"],
    "Hamil Anggur" : ["Kandungan"],
    "Hematuria" : ["Urologi", "Penyakit Dalam", "Kandungan"],
    "Hepatitis" : ["Penyakit Dalam"],
    "Hepatitis A" : ["Penyakit Dalam"],
    "Hepatitis B" : ["Penyakit Dalam"],
    "Hepatitis C" : ["Penyakit Dalam"],
    "Hernia" : ["Bedah"],
    "Herpes Genital" : ["Kulit dan Kelamin"],
    "Herpes Zoster" : ["Kulit dan Kelamin"],
    "Hidrosefalus" : ["Anak"],
    "Hipertensi" : ["Jantung"],
    "Darah Tinggi" : ["Jantung"],
    "Hipertiroidisme" : ["Penyakit Dalam"],
    "Hipoglikemi" : ["Penyakit Dalam"],
    "Hipoglikemia" : ["Penyakit Dalam"],
    "Hipotensi" : ["Jantung"],
    "Darah Rendah" : ["Jantung"],
    "Hipotiroidisme" : ["Penyakit Dalam"],
    "Hiv" : ["Penyakit Dalam"],
    "Hpv" : ["Kandungan", "Kulit dan Kelamin"],
    "Impetigo" : ["Kulit dan Kelamin"],
    "Imunisasi" : ["Anak"],
    "Infeksi Gigi" : ["Gigi"],
    "Infeksi Ginjal" : ["Penyakit Dalam"],
    "Infeksi Jamur" : ["Kulit dan Kelamin"],
    "Infeksi Kulit" : ["Kulit dan Kelamin"],
    "Infeksi Mata" : ["Mata"],
    "Infeksi Rahim" : ["Kandungan"],
    "Infeksi Saluran Kemih" : ["Urologi"],    "Infeksi Salur Kemih" : ["Urologi"],
    "Infeksi Telinga" : ["THT"],
    "Infeksi Telinga Tengah" : ["THT"],
    "Infertilitas" : ["Kandungan"],
    "Insomnia" : ["Saraf", "Jiwa"],
    "Intim Laki-Laki" : ["Kulit dan Kelamin"],  "Intim Laki" : ["Kulit dan Kelamin"],
    "Ispa" : ["Paru", "Anak"],
    "Itp" : ["Penyakit Dalam"],
    "Jerawat" : ["Kulit dan Kelamin"],
    "Kanker Darah" : ["Penyakit Dalam", "Bedah"],
    "Kanker Kulit" : ["Kulit dan Kelamin"],
    "Kanker Paru-Paru" : ["Paru", "Bedah"],   "Kanker Paru" : ["Paru", "Bedah"],
    "Kanker Paru Paru" : ["Paru", "Bedah"],
    "Kanker Payudara" : ["Bedah", "Penyakit Dalam"],
    "Kanker Prostat" : ["Urologi", "Bedah"],
    "Kanker Rahim" : ["Kandungan", "Bedah"],
    "Kanker Serviks" : ["Bedah", "Kandungan"],
    "Kanker Tulang" : ["Tulang", "Bedah"],
    "Kanker Usus Besar" : ["Bedah", "Penyakit Dalam"],
    "Katarak" : ["Mata"],
    "Keguguran" : ["Kandungan"],        "Gugur" : ["Kandungan"],
    "Kehamilan" : ["Kandungan"],        "Hamil" : ["Kandungan"],
    "Kehamilan Ektopik" : ["Kandungan"],  "Hamil Ektopik" : ["Kandungan"],
    "Kelahiran" : ["Kandungan"],         "Lahir" : ["Kandungan"],
    "Kelenjar Getah Bening" : ["THT", "Bedah", "Penyakit Dalam"],
    "Keputihan" : ["Kandungan"],        "Putih" : ["Kandungan"],

    "Keringat" : ["Kulit dan Kelamin"],
    "Kesehatan Mental" : ["Jiwa"],      "Sehat Mental" : ["Jiwa"],
    "Keseleo" : ["Bedah"],
    "Ketombe" : ["Kulit dan Kelamin"],
    "Kista" : ["Kandungan"],
    "Kista Ovarium" : ["Kandungan"],
    "Kolesterol" : ["Jantung", "Penyakit Dalam"],
    "Kolesterol Tinggi" : ["Jantung", "Penyakit Dalam"],
    "Komedo" : ["Kulit dan Kelamin"],
    "Konstipasi" : ["Penyakit Dalam"],
    "Kontrasepsi" : ["Kandungan"],
    "Kram" : ["Tulang"],
    "Kudis" : ["Kulit dan Kelamin"],
    "Kulit" : ["Kulit dan Kelamin"],
    "Kulit Bersisik" : ["Kulit dan Kelamin"],     "Kulit Sisik" : ["Kulit dan Kelamin"],
    "Kurap" : ["Kulit dan Kelamin"],
    "Kutil" : ["Kulit dan Kelamin"],
    "Kutil Kelamin" : ["Kulit dan Kelamin"],
    "Kutu Rambut" : ["Kulit dan Kelamin"],
    "Lensa Kontak" : ["Mata"],
    "Lipoma" : ["Bedah"],
    "Luka Bakar" : ["Bedah"],
    "Lupus" : ["Penyakit Dalam"],
    "Maag" : ["Penyakit Dalam"],
    "Malaria" : ["Penyakit Dalam"],
    "Masturbasi" : ["Kulit dan Kelamin"],
    "Mata Merah" : ["Mata"],
    "Melahirkan" : ["Kandungan"],           "Lahir" : ["Kandungan"],
    "Menopause" : ["Kandungan"],
    "Menstruasi" : ["Kandungan"],
    "Menyusui" : ["Kandungan"],
    "Merencanakan Kehamilan" : ["Kandungan"],   "Rencana Hamil" : ["Kandungan"],
    "Merokok" : ["Paru"],         "Rokok" : ["Paru"],
    "Migrain" : ["Saraf"],
    "Mimisan" : ["THT"],
    "Miom" : ["Kandungan"],
    "Mual" : ["Penyakit Dalam"],
    "Muntah" : ["Penyakit Dalam"],
    "Nutrisi" : ["Gizi"],
    "Nyeri Dada" : ["Jantung"],
    "Nyeri Haid" : ["Kandungan"],


    "Nyeri Sendi" : ["Penyakit Dalam", "Tulang"],
    "Operasi" : ["Bedah"],
    "Orang Tua" : ["Penyakit Dalam"],
    "Osteoarthritis" : ["Tulang"],
    "Osteoporosis" : ["Tulang"],
    "Ovulasi" : ["Kandungan"],
    "Panu" : ["Kulit dan Kelamin"],
    "Pasangan" : ["Kandungan"],       "Pasang" : ["Kandungan"],
    "Patah Tulang" : ["Tulang"],
    "Payudara" : ["Bedah"],
    "Pemutihan Kulit" : ["Kulit dan Kelamin"],              "Putih Kulit" : ["Kulit dan Kelamin"],
    "Penyakit Ginjal" : ["Penyakit Dalam"],                 "Sakit Ginjal" : ["Penyakit Dalam"],
    "Penyakit Jantung" : ["Jantung"],                       "Sakit Jantung" : ["Jantung"],
    "Penyakit Kuning" : ["Penyakit Dalam"],                 "Sakit Kuning" : ["Penyakit Dalam"],
    "Penyakit Mata" : ["Mata"],                             "Sakit Mata" : ["Mata"],
    "Penyakit Menular Seksual" : ["Kulit dan Kelamin"],     "Sakit Tular Seksual" : ["Kulit dan Kelamin"],
    "Penyakit Radang Pinggul" : ["Tulang"],                 "Sakit Radang Pinggul" : ["Tulang"],
    "Perut Kembung" : ["Penyakit Dalam"],
    "Pilek" : ["THT"],
    "Pingsan" : ["Penyakit Dalam", "Jantung", "Saraf"],
    "Plasenta Previa" : ["Kandungan"],
    "Pneumonia" : ["Anak", "Paru"],
    "Polip" : ["THT"],
    "Ppok" : ["Paru"],
    "Psoriasis" : ["Kulit dan Kelamin"],
    "Rabun Jauh" : ["Mata"],
    "Radang Gusi" : ["Gigi"],
    "Radang Usus" : ["Penyakit Dalam"],
    "Rambut" : ["Kulit dan Kelamin"],
    "Rambut Rontok" : ["Kulit dan Kelamin"],
    "Rematik Artritis" : ["Tulang"],
    "Rheumatoid Arthritis" : ["Tulang"],
    "Rhinitis" : ["THT"],
    "Ruam Kulit" : ["Kulit dan Kelamin"],
    "Rubella" : ["Penyakit Dalam"],
    "Sakit Gigi" : ["Gigi"],
    "Sakit Kaki" : ["Tulang"],
    "Sakit Telapak Kaki" : ["Tulang"],
    "Sakit Lutut" : ["Tulang"],
    "Sakit Maag" : ["Penyakit Dalam"],
    "Sakit Lambung" : ["Penyakit Dalam"],
    "Ngilu Pusar" : ["Penyakit Dalam"],
    "Sakit Perut" : ["Penyakit Dalam"],
    "Sakit Pinggang" : ["Penyakit Dalam"],
    "Sakit Pingang" : ["Penyakit Dalam"],
    "Sakit Telinga" : ["THT"],
    "Sakit Tenggorokan" : ["THT"],
    "Hubungan Intim" : ["Kandungan"],       "Hubung Intim" : ["Kandungan"],
    "Serangan Jantung" : ["Jantung"],       "Serang Jantung" : ["Jantung"],
    "Serangan Panik" : ["Jiwa"],            "Serang Panik" : ["Jiwa"],
    "Sesak Napas" : ["Paru"],
    "Sesak Nafas" : ["Paru"],
    "Sifilis" : ["Kulit dan Kelamin"],
    "Sinusitis" : ["THT"],
    "Sirosis" : ["Penyakit Dalam"],
    "Skizofrenia" : ["Jiwa"],
    "Skoliosis" : ["Tulang"],
    "Stres" : ["Jiwa"],
    "Sukralfat" : ["Penyakit Dalam"],
    "Tahi Lalat" : ["Kulit dan Kelamin"],
    "Telinga" : ["THT"],
    "Telinga Berdenging" : ["THT"],         "Telinga Denging" : ["THT"],
    "Tetanus" : ["Penyakit Dalam"],
    "Thalassemia" : ["Anak"],
    "Tifus" : ["Penyakit Dalam"],
    "Tonsilitis" : ["THT"],
    "Trigliserida" : ["Penyakit Dalam"],
    "Trikomoniasis" : ["Kulit dan Kelamin"],
    "Tuberkulosis" : ["Penyakit Dalam"],
    "Tukak Lambung" : ["Penyakit Dalam"],
    "Tumor" : ["Bedah"],
    "Tumor Otak" : ["Saraf", "Bedah"],
    "Tumor Payudara" : ["Bedah"],
    "Usus Buntu" : ["Bedah"],
    "Vagina" : ["Kulit dan Kelamin", "Kandungan"],
    "Vaginitis" : ["Kulit dan Kelamin"],
    "Vaginosis" : ["Kulit dan Kelamin"],
    "Vaksin" : ["Anak"],
    "Vaksinasi" : ["Anak"],
    "Varikokel" : ["Urologi", "Bedah"],
    "Varises" : ["Jantung"],
    "Vitamin A" : ["Mata"],
    "Vitamin C" : ["Kulit dan Kelamin"],
    "Vitamin E" : ["Kulit dan Kelamin"],
    "Vitiligo" : ["Kulit dan Kelamin"],
    "Wajah" : ["Penyakit Dalam"],
    "Wasir" : ["Bedah"],

    'Alzheimer' : ['Saraf', 'Jiwa'],

    'Genetika' : ["Kandungan"],
    'Infeksi' : ["Penyakit Dalam"],
    'Intim' : ['Kandungan'],
    'Intim Laki Laki' : ["Kulit dan Kelamin", "Urologi"],
    'Kanker' : ["Bedah", 'Penyakit Dalam'],
    'Kejang' : ["Saraf"],
    'Meningitis' : ["Penyakit Dalam", "Saraf"],
    'Neuropati' : ['Saraf'],

    'Obesitas' : ['Gizi'],
    'Onkologi' : ["Bedah", 'Penyakit Dalam'],
    'Peningkatan Berat Badan' : ['Gizi'],         'Tingkat Berat Badan' : ['Gizi'],
    'Psikologi Seks Dan Perkawinan' : ['Jiwa'],   'Psikologi Seks Dan Kawin' : ['Jiwa'],
    'Pusing' : ['Saraf'],
    'Radang Otak' : ['Saraf'],
    'Sakit Kepala Tegang' : ['Saraf'],
    'Sariawan' : ['Gigi'],
    'Seks' : ['Kandungan', "Kulit dan Kelamin"],
    'Stroke' : ['Saraf', 'Penyakit Dalam'],
    'Suplemen' : ['Gizi'],
    'Vertigo' : ['THT', 'Saraf'],

    'Acyclovir' : ["Kulit dan Kelamin"],
    'Asam Mefenamat' : ["Gigi", "Kandungan"],

    'Bells Palsy' : ["Saraf"],
    'Cefadroxil' : ["THT", "Urologi", "Kandungan", "Kulit dan Kelamin", "Tulang", "Paru",  "Anak"],
    'Chlamydia' : ["Kulit dan Kelamin"],
    'Ciprofloxacin' : ["Paru", "THT", "Urologi", "Kulit dan Kelamin", "Penyakit Dalam", "Tulang"],
    'Clindamycin' : ["Kulit dan Kelamin", "Kandungan", "Paru", "Tulang", "Penyakit Dalam", "Bedah"],
    'Dexamethasone' : ["Penyakit Dalam", "Tulang", "Kulit dan Kelamin"],

    'Hormon' : ["Kandungan"],
    'Ibuprofen' : ["Gigi", "Tulang", "Kandungan", "Bedah", "Saraf", "Anak"],
    'Impotensi' : ["Urologi"],
    'Kanker Otak' : ["Bedah", 'Saraf'],
    'Kangker Otak' : ["Bedah", 'Saraf'],

    'Kesemutan' : ["Saraf"],          'Semut' : ["Saraf"],
    'Ketoconazole' :  ["Kulit dan Kelamin", "Penyakit Dalam"],
    'Lansoprazole' : ["Penyakit Dalam"],
    'Miconazole' : ["Kulit dan Kelamin"],

    'Omeprazole' : ["Penyakit Dalam"],
    'Paracetamol' : ["Anak", "Kandungan", "Tulang", "Gigi"],
    'Rabies' : ["Penyakit Dalam", "Saraf"],
    'Sakit Leher' : ["Saraf"],
    'Salbutamol' : ["Paru"],



    'Halusinasi'  : ["Jiwa"],
    'Combivent' : ["Paru"] ,
    'Sperma'  : ["Kandungan", "Urologi"],
    'Tenggorokan' : ["THT"],      'Tenggorok' : ["THT"],
    'Bunuh Diri'  : ["Jiwa"],
    'Metabolisme' : ["Gizi"],


    'Kacamata'  : ["Mata"],
    'Susu'  : ["Gizi", "Kandungan"],
    'Bisul' : ["Kulit dan Kelamin"],
    'Sunat' : ["Kulit dan Kelamin"],


    'Nystatin'  : ["Kulit dan Kelamin", "THT", "Gigi"],

    'Berat Badan' : ["Gizi"],
    "Perawatan Muka"  : ["Kulit dan Kelamin"],        "Awat Muka"  : ["Kulit dan Kelamin"],
    "Krim Pagi"  : ["Kulit dan Kelamin"],
    "Krim Malam"  : ["Kulit dan Kelamin"],
    "Prostat" : ["Urologi"],
    "Kadaluarsa"  : ["Gizi"],
    "Sinus" : ["THT"],
    "Hidung Tersumbat"  : ["THT"],            "Hidung Sumbat"  : ["THT"],
    'Rahim' : ["Kandungan"],
    'Pita Suara'  : ["THT"],
    'Tekanan Darah' : ["Jantung", "Penyakit Dalam"],    'Tekan Darah' : ["Jantung", "Penyakit Dalam"],
    'Urin'  : ["Urologi"],
    "Liver" : ["Penyakit Dalam"],
    "Feses" : ["Penyakit Dalam"],
    "Peredaran Darah"  : ["Jantung"],           "Edar Darah"  : ["Jantung"],
    "Flek"  : ["Kulit dan Kelamin", "Kandungan"],
    "Anyang Anyang" : ["Urologi", "Penyakit Dalam"],
    "Ayang" : ["Urologi", "Penyakit Dalam"],
    "Buang Air Kecil" : ["Urologi"],
    "Buang Air Besar" : ["Penyakit Dalam"],
    "Sakit Dada": ["Paru", 'Jantung'],
    "Masalah Jiwa"  : ["Jiwa"],
    "Ganggu Jiwa"  : ["Jiwa"],
    "Endoscopy" : ["Bedah", "Penyakit Dalam"],  "Endoskopi" : ["Bedah", "Penyakit Dalam"],
    "Kecil Paha"  : ["Gizi"],
    "Buncit"  : ["Gizi"],
    "Sakit Kepala" : ["Saraf"],
    "Ansietas"  : ["Jiwa"],   "Cemas" : ["Jiwa"],   "Anxiety" : ["Jiwa"],
    "Bab" : ["Penyakit Dalam"],
    "Muda Marah"  : ["Jiwa"],
    "Belakang Mental"  : ["Jiwa"],
    "Inseminasi"  : ["Kandungan"],
    "Kaki Dingin" : ["Jiwa"],
    "Gemuk" : ["Gizi"],
    "Cacar" : ["Kulit dan Kelamin"],
    "Selalu Kencing" : ["Urologi"],
    'Bunions' : ["Bedah", "Tulang"],
    'Perih Bak' : ["Penyakit Dalam", "Bedah", 'Urologi'],
    'Perih Kencing' : ["Penyakit Dalam", "Bedah", 'Urologi'],
    'Sakit Bak' : ["Penyakit Dalam", "Bedah", 'Urologi'],
    'Bak Sakit' : ["Penyakit Dalam", "Bedah", 'Urologi'],
    'Telat Datang Bulan'  : ["Kandungan"],
    'Imunitas'  : ['Gizi'],
    'Kencing Sedikit' : ['Urologi'],
    'Ulu Hati'  :   ['Bedah', 'Penyakit Dalam'],
    'Hulu Hati'  :   ['Bedah', 'Penyakit Dalam'],
    'Sakit Tel' : ['Gigi', 'THT'],
    'Punggung Sakit'  : ['Tulang', 'Saraf'],
    'Sakit Badan' : ['Tulang', 'Saraf'],
    'Linu'  : ['Tulang', 'Penyakit Dalam'],
    'Sakit Malu'  : ["Kulit dan Kelamin", 'Urologi'], # Sakit Kemaluan

    'Buah Zakar'  : ['Urologi', 'Kulit dan Kelamin'],
    'Bersin'  : ['THT'],
    'Suara Hilang'  : ['THT'],
    'Lulur'   : ['Kulit dan Kelamin'],
    'Kentut'  :   ['Penyakit Dalam'],
    'Urethral'  : ['Bedah', 'Penyakit Dalam'],
    'Sesak Dada'  : ['Paru'],
    'Hipokondria' : ['Jiwa'],
    'Dada Sakit'  : ['Paru', 'Jantung'],
    'Suntik Silikon'  : ['Kulit dan Kelamin'],
    'Berhubngan Suami' : ["Kandungan"],
    'Otot Lengan' : ['Tulang'],
    'Khayal'  : ['Jiwa'],
    'Datang Bulan'  : ['Kandungan'],
    'Lhran' : ['Kandungan'],  # Lahiran
    'Rproduksi' : ['Kandungan'],
    'Makan' : ['Gizi'],
    'Kelamin Lelaki'  : ['Kulit dan Kelamin', 'Urologi'],

    'Disorder'  : ['Jiwa'],
    'Micin Bodoh' : ['Gizi'],
    'Tinggi Badan'  : ['Gizi'],
    'Buka Mulut'  : ['Gigi'],
    'Fobia' : ['Jiwa'],
    'Rongga Mulut'  : ['Gigi'],
    'Cair Hidung' : ['THT'],
    'Badan Kurus' : ['Gizi'],
    'Sgot'  : ['Penyakit Dalam'],
    'Sgpt'  : ['Penyakit Dalam'],
    'Anosmia' : ['THT'],
    'Kalori'  : ['Gizi'],
    'Enzim Hati'  : ['Penyakit Dalam'],
    'Sel Telur' : ['Kandungan'],
    'Hitam Mulut' : ['Gigi'],
    'Hantu' : ['Jiwa'],
    'Sering Lupa'  : ['Saraf'],
    'Gusi Darah'  : ['Gigi'],
    'Rahang'  : ['Gigi'],
    'Firusa'  : ['Bedah'],        'Fisura'  : ['Bedah'],

    'Sulit Ingat' : ['Saraf'],
    "Bumbu Masak" : ['Gizi'],
    'Mancung Hidung'  : ['Bedah'],
    'Bindeng' : ['THT'],
    'Hilang Bulu' : ['Kulit dan Kelamin'],
    'Keloid'  : ['Bedah', 'Kulit dan Kelamin'],
    'Kb'  : ['Kandungan'],
    'Kandung Kemih' : ['Urologi'],
    'Neurogenik'  : ['Saraf'],
    'Janin' : ['Kandungan'],
    'Sulit Nafas' : ['THT'],
    'Hbv' : ['Penyakit Dalam'],
    'Kelamin' : ['Urologi', 'Kulit dan Kelamin'],
    'Mie Instan'  : ['Gizi'],
    'Salur Kemih' : ['Urologi'],
    'Sedih' : ['Jiwa'],
    'Kandung Kemih' : ['Urologi'],
    'Cholelithiasis'  : ['Bedah'],
    'Tubuh Kurus' : ['Gizi'],
    'bintik bintik' : ['Kulit dan Kelamin'],
    'nyeri' : ['Penyakit Dalam'],
    'mr p' : ['Kulit dan Kelamin'],
    'erythromycin' : ['Kulit dan Kelamin', 'Penyakit Dalam', 'Urologi'],
    'promag' : ['Penyakit Dalam'],
    'jari kaki kaku' : ['Penyakit Dalam'],
    'thorax' : ['Paru'],
    'dahak ada darah' : ['THT', 'Penyakit Dalam'],
    'celaka motor' : ['Penyakit Dalam', 'Tulang'],
    'insulin' : ['Penyakit Dalam'],
    'shabu' : ['Jiwa'],
    'filler hidung' : ['Kulit dan Kelamin'],
    'biasa onani' : ['Jiwa'],
    'ligamen' : ['Tulang'],
    'tendon' : ['Tulang'],
    'ganggu lihat' : ['Mata'],
    'ludah campur darah' : ['THT', 'Penyakit Dalam'],
    'tanam benang' : ['Kulit dan Kelamin'],
    'lepas iud' : ['Kandungan'],
    'Buduk' : ['Kulit dan Kelamin'],
    'cairanx pnuh v' : ["Kandungan"],
    'manusia paling dosa' : ["Jiwa"]

}

## Mapper

---



In [28]:
df_corpus["CLASS_JUDUL"] = [mapping(s) for s in df_corpus["JUDUL"]]
df_corpus["CLASS_ISI"] = [mapping(s) for s in df_corpus["ISI"]]
df_corpus["CLASS_JUDUL_ISI"] = [mapping((j + " SEP " + i)) for j, i in df_corpus[["JUDUL", "ISI"]].values]
df_corpus["NEW_CLASS"] = [mapping(" , ".join(s)) for s in df_corpus["CLASS"]]

In [29]:
df_corpus.head(3)

,CLASS,JUDUL,ISI,JAWABAN,CLASS_JUDUL,CLASS_ISI,CLASS_JUDUL_ISI,NEW_CLASS
ID,,,,,,,,
DS-1,[other],mengapa keringat badan lebih,tanya konsultasi selamat malam dokter fathurro...,selamat malam Fathurrosi. Maaf berapa berat ba...,[Kulit dan Kelamin],[Kulit dan Kelamin],[Kulit dan Kelamin],[]
DS-23,[other],suka menghayal halusinasi,tanya konsultasi selamat siang dokter kali kha...,"Bila berkelanjutan, bisa saja memiliki ganggua...",[Jiwa],[Jiwa],[Jiwa],[]
DS-87,[other],feses warna kuning serta darah apakah ada peny...,tanya konsultasi selamat sore dokter feses war...,Kemungkinan kamu mengalami wasir atau ambeien....,[Penyakit Dalam],[Penyakit Dalam],[Penyakit Dalam],[]


In [30]:
var = "dok , saya seorang pria berusia 28 tahun dan bekerja sebagai pengajar . saya ingin menceritakan terlebih dahulu kronologis kejadian yang membuat saya mengalami penyakit ini . sekitar satu setengah bulan yang lalu saya mengalami tekanan darah tinggi dengan hasil pengukuran 180 /100 mmhg . pada saat itu saya kerja lembur sampai sore dan dilanjutkan mengikuti kegiatan di desa saya tanpa sempat makan dan istirahat . perawat yang memeriksa saya mengatakan kemungkinan tekanan darah saya tinggi karena terlalu capai dan stress . gejala yang saya rasakan pada saat itu adalah debar jantung yang sangat cepat , rasa mual , tengkuk terasa kaku , dan kepala pusing . pikiran saya menjadi tambah kacau dan debar jantung semakin cepat karena saya teringat dengan kejadian pada waktu ibu saya mengalami pendarahan batang otak akibat pecahnya pembuluh darah karena hipertensi ( saya melihat langsung kejadiannya dari awal sampai beliau masuk rumah sakit ) . saya berusaha untuk tetap sadar walaupun rasanya seperti mau pingsan dengan harapan agar kejadian yang menimpa ibu saya tidak terjadi pada saya . setelah saya diberi obat penurun tekanan darah , selang satu jam tekanan darah saya berangsur turun dan dapat beristirahat . setelah kejadian pertama tersebut saya memutuskan untuk membeli tensimeter digital untuk mengecek tekanan darah setiap hari . hasilnya selama tiga minggu tekanan darah saya normal ( antara 110 125 / 70 80 mmhg ) . kemudian pada waktu ada acara di tempat kerja , gejala detak jantung cepat saya rasakan kembali tetapi tidak disertai tengkuk kaku , melainkan nafas terasa agak sesak . saya kemudian minum obat penurun tekanan darah tetapi selang dua jam detak jatung saya semakin bertambah cepat karena teringat kejadian pertama . kemudian saya dibawa ke rumah sakit dan menjalani serangkaian tes , yaitu pengukuran tekanan darah , rekam jantung , dan cek gula darah . hasilnya dokter mengatakan hasil rekam jantung saya bagus , gula darah normal tetapi tekanan darah 170 /100 mmhg . selanjutnya saya diberi obat penurun tekanan darah , oksigen , dan suntik untuk mengurangi asam lambung . dua jam kemudian saya diperbolehkan pulang dan diberi tiga macam obat yang masing-masing berguna untuk penurun tekanan darah , mengurangi asam lambung , dan pereda nyeri ( berdasarkan pencarian di internet dari nama obatnya ) . setelah kejadian kedua tersebut , tekanan darah saya selalu normal . tetapi dalam waktu dua minggu ini saya sering mengalami gejala yang saya anggap agak aneh . setiap berada di sekolah pasti gejala aneh ini terjadi , tetapi tidak demikian jika berada di rumah . gejala yang saya rasakan adalah ketika saya merasa sedikit sesak nafas ( mungkin akibat rokok atau berada di ruang tertutup agak lama ) secara spontan saya teringat dengan dua kejadian di atas . karena bertambah cemas rasa sesak nafas ini semakin menjadi diikuti detak jantung cepat , tubuh terasa kesemutan , sering buang air kecil , tidak tenang , kepala berat tetapi tidak pusing , mulut kering dan terasa asam , dan agak susah menelan . ketika diukur berulang kali , tekanan darah saya maksimal hanya mencapai 138 /82 mmhg . gejala ini dapat bertahan selama beberapa jam . bahkan dalam perjalanan pulang dada semakin bertambah sesak dan pikiran tidak fokus . terakhir kali gejala ini bisa hilang selama perjalanan pulang pada saat saya dapat mengingat hal-hal menyenangkan . tetapi kalau sedang berhenti di lampu merah dan pikiran tidak fokus , gejala sesak nafas langsung muncul kembali dan seterusnya sampai di rumah . anehnya , ketika sampai di rumah gejala ini kemudian hilang dan tidak pernah terjadi di rumah . munculnya gejala ini sudah saya alami 3 4 kali dalam waktu dua minggu ini . pertanyaan saya : 1 . menurut dokter , penyakit apa yang sebenarnya ada dalam diri saya ? apakah hipertensi atau penyakit lain yang berhubungan dengan psikologis ? 2 . dari hasil membaca artikel saya mengenal penyakit anxiety disorder ? . apakah saya mengalami penyakit ini ? 3 . apa saja yang harus saya lakukan untuk mengobati penyakit tersebut ? mohon maaf jika uraian saya terlalu panjang . mohon bantuannya karena penyakit ini benar-benar mengganggu aktivitas saya . terima kasih sebelumnya ."

var = stemmer.stem(stopword.remove(re.sub(r'[^\w]', ' ', str(var).lower())))
mapping(var)

['Paru', 'Jiwa', 'Saraf', 'Urologi', 'Penyakit Dalam', 'Gizi', 'Jantung']

In [31]:
print("Jumlah instances CLASS_JUDUL yang bukan null: " + str(len(df_corpus[df_corpus["CLASS_JUDUL"].str.len() > 0])))
print("Jumlah instances CLASS_JUDUL yang null: " + str(len(df_corpus[df_corpus["CLASS_JUDUL"].str.len() == 0])))
print("Jumlah instances CLASS_ISI yang bukan null: " + str(len(df_corpus[df_corpus["CLASS_ISI"].str.len() > 0])))
print("Jumlah instances CLASS_ISI yang null: " + str(len(df_corpus[df_corpus["CLASS_ISI"].str.len() == 0])))
print("Jumlah instances CLASS_JUDUL_ISI yang bukan null: " + str(len(df_corpus[df_corpus["CLASS_JUDUL_ISI"].str.len() > 0])))
print("Jumlah instances CLASS_JUDUL_ISI yang null: " + str(len(df_corpus[df_corpus["CLASS_JUDUL_ISI"].str.len() == 0])))

Jumlah instances CLASS_JUDUL yang bukan null: 1415
Jumlah instances CLASS_JUDUL yang null: 144
Jumlah instances CLASS_ISI yang bukan null: 1506
Jumlah instances CLASS_ISI yang null: 53
Jumlah instances CLASS_JUDUL_ISI yang bukan null: 1559
Jumlah instances CLASS_JUDUL_ISI yang null: 0


# Compare Metric

In [32]:
from sklearn.metrics import classification_report, accuracy_score, jaccard_score, \
hamming_loss, cohen_kappa_score, f1_score, precision_score, recall_score
import math
def eval_all(y_human, y_machine, mlb):
  print("================ Classification Report ================")
  print(classification_report(y_human, y_machine, target_names=mlb.classes_, digits=4))
  print("================ Multi Label Score ================")
  eval_multi_label(y_human, y_machine, mlb)
  print("================ Cohen Kappa ================")
  cohen_kappa(y_human, y_machine, mlb)

def eval_multi_label(y_test, prediction, mlb):
    # Butuh informasi lebih lengkap? silakan disimak di bawah ini
    print('Accuracy             :', accuracy_score(y_test, prediction))
    print('Precision            :', precision_score(y_test, prediction, average='micro'))
    print('Recall               :', recall_score(y_test, prediction, average='micro'))
    print('F1 Score             :', f1_score(y_test, prediction, average='micro'))
    print('Hamming Loss         :', hamming_loss(y_test, prediction))
    print('Jaccard Score')
    js = jaccard_score(y_test, prediction, average=None)
    for i, j in zip(mlb.classes_, js):
      print(f"{i:<20} : {j}")
    print('Jaccard Score Macro Average:', jaccard_score(y_test, prediction, average='macro'))
    print('Jaccard Score Micro Average:', jaccard_score(y_test, prediction, average='micro'))

def cohen_kappa(y_human, y_test, mlb):
  sum = 0
  count = 0
  for i in range(len(mlb.classes_)):
    score = cohen_kappa_score(y_human[:,i], y_test[:,i])
    if not math.isnan(score):
      sum += score
      count += 1
    print(f"Cohen Kappa {mlb.classes_[i]:<20} : {cohen_kappa_score(y_human[:,i], y_test[:,i])}")
  print("Average Cohen Kappa              : " + str(sum/count))

## By Metadata

In [33]:
df = pd.merge(df_corpus.drop(columns=["JUDUL", "ISI", "JAWABAN", "CLASS_JUDUL", "CLASS_ISI", "CLASS_JUDUL_ISI", "CLASS"]),
                       df_human.drop(columns=["JUDUL", "ISI"]),
                       on="ID")
class_col = "NEW_CLASS"
df = df[df['NEW_CLASS'].map(lambda d: len(d)) > 0]
df

,NEW_CLASS,Anak,Bedah,Gigi,Gizi,Jantung,Jiwa,Kandungan,Kulit dan Kelamin,Mata,Paru,Penyakit Dalam,Saraf,THT,Tulang,Urologi
ID,,,,,,,,,,,,,,,,
AD-27,[Gizi],0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
AD-47,[Kandungan],0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
AD-117,"[Bedah, Penyakit Dalam]",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
AD-148,"[Bedah, Kulit dan Kelamin, Penyakit Dalam]",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
AD-179,"[Penyakit Dalam, Kandungan]",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KD-43177,[Penyakit Dalam],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
KD-43189,[THT],0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
KD-43201,[Bedah],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [34]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb_std = MultiLabelBinarizer()
mlb_std.fit(df[class_col].values)

MultiLabelBinarizer()

In [35]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
y_human = df.drop(columns=[class_col]).values
y_machine = mlb_std.transform(df[class_col].values)

In [36]:
len(y_machine)

1265

In [37]:
eval_all(y_human, y_machine, mlb_std)

================ Classification Report ================
                   precision    recall  f1-score   support

             Anak     0.7518    0.7923    0.7715       130
            Bedah     0.4235    0.6154    0.5017       117
             Gigi     0.7838    0.8529    0.8169        34
             Gizi     0.6556    0.5728    0.6114       103
          Jantung     0.1500    0.2727    0.1935        22
             Jiwa     0.8095    0.7816    0.7953        87
        Kandungan     0.8095    0.8327    0.8209       245
Kulit dan Kelamin     0.6919    0.8538    0.7644       171
             Mata     1.0000    0.8986    0.9466        69
             Paru     0.5200    0.7222    0.6047        18
   Penyakit Dalam     0.4683    0.7711    0.5827       201
            Saraf     0.3111    0.2857    0.2979        49
              THT     0.8633    0.8759    0.8696       137
           Tulang     0.5208    0.5952    0.5556        42
          Urologi     0.1250    0.1818    0.1481        33

## By Judul

In [38]:
df = pd.merge(df_corpus.drop(columns=["JUDUL", "ISI", "JAWABAN", "CLASS_ISI", "CLASS_JUDUL_ISI", "CLASS", "NEW_CLASS"]),
                       df_human.drop(columns=["JUDUL", "ISI"]),
                       on="ID")
df = df[df["CLASS_JUDUL"].str.len() > 0]
class_col = "CLASS_JUDUL"
df

,CLASS_JUDUL,Anak,Bedah,Gigi,Gizi,Jantung,Jiwa,Kandungan,Kulit dan Kelamin,Mata,Paru,Penyakit Dalam,Saraf,THT,Tulang,Urologi
ID,,,,,,,,,,,,,,,,
DS-1,[Kulit dan Kelamin],0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
DS-23,[Jiwa],0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
DS-87,[Penyakit Dalam],0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
DS-176,"[Bedah, Penyakit Dalam]",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
DS-324,[Bedah],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KD-43177,[Bedah],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
KD-43189,[THT],0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
KD-43201,[Bedah],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [39]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
y_human = df.drop(columns=[class_col]).values
y_machine = mlb_std.transform(df[class_col].values)

In [40]:
eval_all(y_human, y_machine, mlb_std)

================ Classification Report ================
                   precision    recall  f1-score   support

             Anak     0.7516    0.6722    0.7097       180
            Bedah     0.3933    0.6861    0.5000       137
             Gigi     0.8684    0.9429    0.9041        35
             Gizi     0.4189    0.8942    0.5706       104
          Jantung     0.4000    0.7778    0.5283        36
             Jiwa     0.9355    0.6374    0.7582        91
        Kandungan     0.6482    0.8667    0.7417       270
Kulit dan Kelamin     0.6098    0.8794    0.7202       199
             Mata     0.9125    0.9733    0.9419        75
             Paru     0.3704    0.6250    0.4651        16
   Penyakit Dalam     0.5000    0.6509    0.5656       212
            Saraf     0.4242    0.5600    0.4828        50
              THT     0.8986    0.8552    0.8763       145
           Tulang     0.6500    0.6341    0.6420        41
          Urologi     0.3913    0.5294    0.4500        34

## By Isi

In [41]:
df = pd.merge(df_corpus.drop(columns=["JUDUL", "ISI", "JAWABAN", "CLASS_JUDUL", "CLASS_JUDUL_ISI", "CLASS", "NEW_CLASS"]),
                       df_human.drop(columns=["JUDUL", "ISI"]),
                       on="ID")
df = df[df["CLASS_ISI"].str.len() > 0]
class_col = "CLASS_ISI"
df

,CLASS_ISI,Anak,Bedah,Gigi,Gizi,Jantung,Jiwa,Kandungan,Kulit dan Kelamin,Mata,Paru,Penyakit Dalam,Saraf,THT,Tulang,Urologi
ID,,,,,,,,,,,,,,,,
DS-1,[Kulit dan Kelamin],0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
DS-23,[Jiwa],0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
DS-87,[Penyakit Dalam],0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
DS-176,"[Bedah, Penyakit Dalam]",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
DS-277,"[Saraf, Paru, Jiwa]",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KD-43177,"[Bedah, Kandungan]",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
KD-43189,"[THT, Jantung]",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
KD-43201,[Bedah],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [42]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
y_human = df.drop(columns=[class_col]).values
y_machine = mlb_std.transform(df[class_col].values)

In [43]:
eval_all(y_human, y_machine, mlb_std)

================ Classification Report ================
                   precision    recall  f1-score   support

             Anak     0.5856    0.9448    0.7230       181
            Bedah     0.3274    0.7817    0.4615       142
             Gigi     0.5469    0.9722    0.7000        36
             Gizi     0.2409    0.9815    0.3869       108
          Jantung     0.2536    0.9459    0.4000        37
             Jiwa     0.7551    0.6981    0.7255       106
        Kandungan     0.5228    0.9259    0.6683       297
Kulit dan Kelamin     0.4773    0.9310    0.6311       203
             Mata     0.6466    0.9740    0.7772        77
             Paru     0.2055    0.7500    0.3226        20
   Penyakit Dalam     0.3352    0.7448    0.4623       239
            Saraf     0.1944    0.5738    0.2905        61
              THT     0.6919    0.9073    0.7851       151
           Tulang     0.3400    0.7391    0.4658        46
          Urologi     0.2763    0.5676    0.3717        37

## By Judul + Isi

In [44]:
df = pd.merge(df_corpus.drop(columns=["JUDUL", "ISI", "JAWABAN", "CLASS_JUDUL", "CLASS_ISI", "CLASS", "NEW_CLASS"]),
                       df_human.drop(columns=["JUDUL", "ISI"]),
                       on="ID")
df = df[df["CLASS_JUDUL_ISI"].str.len() > 0]
class_col = "CLASS_JUDUL_ISI"
df

,CLASS_JUDUL_ISI,Anak,Bedah,Gigi,Gizi,Jantung,Jiwa,Kandungan,Kulit dan Kelamin,Mata,Paru,Penyakit Dalam,Saraf,THT,Tulang,Urologi
ID,,,,,,,,,,,,,,,,
DS-1,[Kulit dan Kelamin],0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
DS-23,[Jiwa],0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
DS-87,[Penyakit Dalam],0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
DS-176,"[Bedah, Penyakit Dalam]",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
DS-277,"[Saraf, Paru, Jiwa]",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KD-43177,"[Bedah, Kandungan]",0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
KD-43189,"[THT, Jantung]",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
KD-43201,[Bedah],0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [45]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
y_human = df.drop(columns=[class_col]).values
y_machine = mlb_std.transform(df[class_col].values)

In [46]:
eval_all(y_human, y_machine, mlb_std)

================ Classification Report ================
                   precision    recall  f1-score   support

             Anak     0.5812    0.9676    0.7262       185
            Bedah     0.3197    0.8069    0.4579       145
             Gigi     0.5538    0.9730    0.7059        37
             Gizi     0.2428    0.9910    0.3901       111
          Jantung     0.2587    0.9737    0.4088        38
             Jiwa     0.7642    0.7232    0.7431       112
        Kandungan     0.5249    0.9344    0.6722       305
Kulit dan Kelamin     0.4733    0.9488    0.6316       215
             Mata     0.6555    0.9750    0.7839        80
             Paru     0.2179    0.8095    0.3434        21
   Penyakit Dalam     0.3316    0.7724    0.4640       246
            Saraf     0.1958    0.5873    0.2937        63
              THT     0.7024    0.9231    0.7978       156
           Tulang     0.3462    0.7500    0.4737        48
          Urologi     0.3103    0.6923    0.4286        39

# Prediksi

In [47]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import re
import random
import seaborn as sns
import string
from collections import Counter
import warnings
warnings.filterwarnings("ignore", 'This pattern has match groups')

from tqdm import tqdm

import nltk
from nltk.tokenize import word_tokenize
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.probability import FreqDist
nltk.download('punkt')

from xgboost import XGBClassifier

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score, f1_score, hamming_loss

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [48]:
seed_val = 1906350912
random.seed(seed_val)
np.random.seed(seed_val)
categories = ['Anak', 'Bedah', 'Gigi', 'Gizi', 'Jantung', 'Jiwa',
       'Kandungan', 'Kulit dan Kelamin', 'Mata', 'Paru', 'Penyakit Dalam',
       'Saraf', 'THT', 'Tulang', 'Urologi']

categories_umum = ['Anak', 'Bedah', 'Gigi', 'Gizi', 'Jantung', 'Jiwa',
       'Kandungan', 'Kulit dan Kelamin', 'Mata', 'Paru', 'Penyakit Dalam',
       'Saraf', 'THT', 'Tulang', 'Umum', 'Urologi']
# torch.manual_seed(seed_val)

In [49]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit_transform([categories])

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [50]:
df_test = pd.read_csv(f"Dataset/gold_standard.csv", index_col="ID").drop(columns=["Unnamed: 0"])
df_test.fillna("", inplace = True)
df_test['Count'] = df_test.drop(columns=['JUDUL', 'ISI']).values.sum(axis=1)
df_test = df_test[df_test['Count'] <= 3].drop(columns=['Count'])
df_test = df_test[["JUDUL", "ISI"] + categories_umum]

df_test_umumnt = df_test.drop(columns=["Umum"])
df_test_umumnt["COUNT CLASS"] = df_test_umumnt.drop(columns=["JUDUL", "ISI"]).sum(axis=1)
df_test = df_test_umumnt[df_test_umumnt["COUNT CLASS"] > 0].drop(columns=["COUNT CLASS"])
del df_test_umumnt

In [51]:
import re
df_test["JUDUL"] = [re.sub(r'[^\w]', ' ', str(s).lower()) for s in df_test["JUDUL"].values]
df_test["ISI"] = [re.sub(r'[^\w]', ' ', str(s).lower()) for s in df_test["ISI"].values]

In [52]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
factory = StopWordRemoverFactory()
# stopwords = factory.get_stop_words()
# ext_stopwords = ['dok', 'doc', 'dokter', 'terima', 'kasih', 'terimakasih']
# whitelist = ['kecil', 'besar', 'mata', 'masalah', 'belakang']
# all_stopwords = list((set(stopwords) | set(ext_stopwords)) - set(whitelist))
all_stopwords = ["di", 'pada', 'ingin', 'dan', 'yang', 'sangat', 'luar', 'biasa', 'bagi', 'bagian', 'saat', 'hanya', 'asa', 'terasa', 'saat', 'dengan', 'bikin', 'dari', 'dalam', 'sering', 'saya', 'untuk']
dictionary = ArrayDictionary(all_stopwords)
stopword = StopWordRemover(dictionary)
df_test["JUDUL"] = [stopword.remove(s) for s in df_test["JUDUL"]]
df_test["ISI"] = [stopword.remove(s) for s in df_test["ISI"]]

In [53]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()
ignore = ["menyusui"]
# [" ".join([stemmer.stem(w) if w not in ignore else w for w in s.split()]) for s in lst])]
df_test["JUDUL"] = [" ".join([stemmer.stem(w) if w not in ignore else w for w in str(s).split()]) for s in df_test["JUDUL"]]
df_test["ISI"] = [" ".join([stemmer.stem(w) if w not in ignore else w for w in str(s).split()]) for s in df_test["ISI"]]

In [54]:
df_test["CLASS_JUDUL"] = [mapping(s) for s in df_test["JUDUL"]]
df_test["CLASS_ISI"] = [mapping(s) for s in df_test["ISI"]]
df_test["CLASS_JUDUL_ISI"] = [mapping((j + " SEP " + i)) for j, i in df_test[["JUDUL", "ISI"]].values]

In [55]:
print("Jumlah instances CLASS_JUDUL yang bukan null: " + str(len(df_test[df_test["CLASS_JUDUL"].str.len() > 0])))
print("Jumlah instances CLASS_JUDUL yang null: " + str(len(df_test[df_test["CLASS_JUDUL"].str.len() == 0])))
print("Jumlah instances CLASS_ISI yang bukan null: " + str(len(df_test[df_test["CLASS_ISI"].str.len() > 0])))
print("Jumlah instances CLASS_ISI yang null: " + str(len(df_test[df_test["CLASS_ISI"].str.len() == 0])))
print("Jumlah instances CLASS_JUDUL_ISI yang bukan null: " + str(len(df_test[df_test["CLASS_JUDUL_ISI"].str.len() > 0])))
print("Jumlah instances CLASS_JUDUL_ISI yang null: " + str(len(df_test[df_test["CLASS_JUDUL_ISI"].str.len() == 0])))

Jumlah instances CLASS_JUDUL yang bukan null: 2321
Jumlah instances CLASS_JUDUL yang null: 391
Jumlah instances CLASS_ISI yang bukan null: 2532
Jumlah instances CLASS_ISI yang null: 180
Jumlah instances CLASS_JUDUL_ISI yang bukan null: 2636
Jumlah instances CLASS_JUDUL_ISI yang null: 76


In [56]:
y_pred = mlb.transform(df_test["CLASS_JUDUL_ISI"].values)

In [57]:
y_test = df_test[categories].values

In [58]:
eval_all(y_test, y_pred, mlb)

================ Classification Report ================
                   precision    recall  f1-score   support

             Anak     0.6032    0.9196    0.7285       286
            Bedah     0.3896    0.7300    0.5081       237
             Gigi     0.7702    0.8377    0.8025       228
             Gizi     0.3434    0.9038    0.4977       239
          Jantung     0.3503    0.8611    0.4980        72
             Jiwa     0.7917    0.5278    0.6333       144
        Kandungan     0.4965    0.9084    0.6421       393
Kulit dan Kelamin     0.5586    0.7733    0.6486       419
             Mata     0.7023    0.9767    0.8171       215
             Paru     0.1000    0.5789    0.1705        19
   Penyakit Dalam     0.3586    0.7230    0.4794       426
            Saraf     0.2792    0.7053    0.4000        95
              THT     0.6095    0.8766    0.7190       235
           Tulang     0.4855    0.6364    0.5508       132
          Urologi     0.4825    0.4365    0.4583       126

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [59]:
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='micro', zero_division=0)
hamming = hamming_loss(y_test, y_pred)

print(f'Accuracy: {acc}')
print(f'F1 (Micro): {f1}')
print(f'Hamming Loss: {hamming}')
print(classification_report(y_test, y_pred, target_names=categories, zero_division=0))

Accuracy: 0.2935103244837758
F1 (Micro): 0.5964711274060496
Hamming Loss: 0.08657817109144543
                   precision    recall  f1-score   support

             Anak       0.60      0.92      0.73       286
            Bedah       0.39      0.73      0.51       237
             Gigi       0.77      0.84      0.80       228
             Gizi       0.34      0.90      0.50       239
          Jantung       0.35      0.86      0.50        72
             Jiwa       0.79      0.53      0.63       144
        Kandungan       0.50      0.91      0.64       393
Kulit dan Kelamin       0.56      0.77      0.65       419
             Mata       0.70      0.98      0.82       215
             Paru       0.10      0.58      0.17        19
   Penyakit Dalam       0.36      0.72      0.48       426
            Saraf       0.28      0.71      0.40        95
              THT       0.61      0.88      0.72       235
           Tulang       0.49      0.64      0.55       132
          Urologi   